<a href="https://colab.research.google.com/github/eloveman/Homework/blob/main/HWneural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
pip install ucimlrepo

In [56]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from ucimlrepo import fetch_ucirepo

# fetch dataset
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# data (as pandas dataframes)
X = cdc_diabetes_health_indicators.data.features
y = cdc_diabetes_health_indicators.data.targets

# metadata
print(cdc_diabetes_health_indicators.metadata)

# variable information
print(cdc_diabetes_health_indicators.variables)

{'uci_id': 891, 'name': 'CDC Diabetes Health Indicators', 'repository_url': 'https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators', 'data_url': 'https://archive.ics.uci.edu/static/public/891/data.csv', 'abstract': 'The Diabetes Health Indicators Dataset contains healthcare statistics and lifestyle survey information about people in general along with their diagnosis of diabetes. The 35 features consist of some demographics, lab test results, and answers to survey questions for each patient. The target variable for classification is whether a patient has diabetes, is pre-diabetic, or healthy. ', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 253680, 'num_features': 21, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Sex', 'Age', 'Education Level', 'Income'], 'target_col': ['Diabetes_binary'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_

In [57]:
if isinstance(X, pd.DataFrame):  #I had a hard time figuring out that the data set was already split
    print("df is a valid DataFrame.")
else:
    print("df is not a DataFrame.")

df is a valid DataFrame.


Part 1: Feature Selection

In [58]:
selected_columns = ['BMI', 'Sex', 'HvyAlcoholConsump', 'PhysHlth', 'Education', 'Income']
data_model = X[selected_columns]

#I choose BMI because people with type 2 diabetes tend to have a high BMI, sex because it would be intresting to see if there is a difference.
#Heavy Alchol Consumption because alchol is high in sugar and high sugar leads to diabetes
#PhysHlth people that are healthy should have a lower BMI and chance of getting type 2 diabetes.
#Education because I think that people that learn that eating healthy and exercising is important will
#Income because people with a higher income can afford healthy food.


Part 2: Data Cleaning

In [59]:
#Figuring out what are the numeric columns in the X dataset

data_types = X.dtypes
numeric_cols = data_types[data_types.apply(lambda x: pd.api.types.is_numeric_dtype(x))].index
print("Numeric Columns:")
print(numeric_cols)
numeric_features = X[numeric_cols]

Numeric Columns:
Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')


In [60]:
#Figuring out if Diabetes_binary is numeric (common sense says that it is but just checking)
data_types = y.dtypes
numeric_cols = data_types[data_types.apply(lambda x: pd.api.types.is_numeric_dtype(x))].index
print("Numeric Columns:")
print(numeric_cols)
numeric_features = y[numeric_cols]

Numeric Columns:
Index(['Diabetes_binary'], dtype='object')


In [61]:
#Normalizing the numeric features

numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()

# Fit and transform the scaler on X data frame
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])



<ipython-input-61-5c92877e04db>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_cols] = scaler.fit_transform(X[numeric_cols])


In [62]:
# Remove outliers from X and y
from scipy import stats
import numpy as np

z_scores = stats.zscore(X[numeric_cols])
abs_z_scores = np.abs(z_scores)
outliers = (abs_z_scores > 3).all(axis=1)

X = X[~outliers]
y = y[~outliers]

Part 3: Feature Engineering


In [63]:
# I am in a sociology class and we talk about SES, Socioeconomic status so for my engineered variable I am going to
#make this SES variable by combining, sex, education, and Income

X['SES_Variable'] = X['Sex'] * X['Education']* X['Income']


In [64]:
from sklearn.model_selection import train_test_split

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Looking to see what the data looks like
X_train.describe()


,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,SES_Variable
count,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,...,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000,202944.000000
mean,-0.001595,-0.000925,0.000416,-0.001588,-0.000609,0.000035,0.000179,0.001251,0.000313,0.000884,...,-0.000252,0.000410,0.001072,0.000769,0.000211,0.000171,0.000232,0.000189,-0.000610,-0.043498
std,0.999772,0.999860,0.998987,1.000560,0.999933,1.000084,1.000251,0.999253,0.999915,0.999298,...,0.999625,0.999780,1.001863,1.000341,1.000189,1.000023,0.999630,1.000947,1.000579,1.174164
min,-0.866785,-0.858182,-5.078164,-2.478916,-0.892119,-0.205637,-0.322458,-1.762814,-1.316872,-2.074316,...,-0.303173,-1.414532,-0.429630,-0.486592,-0.449718,-0.887021,-2.302431,-4.108894,-2.440138,-8.893508
25%,-0.866785,-0.858182,0.196922,-0.663122,-0.892119,-0.205637,-0.322458,0.567275,-1.316872,0.482087,...,-0.303173,-0.478619,-0.429630,-0.486592,-0.449718,-0.887021,-0.665349,-1.065595,-0.508837,-0.552578
50%,-0.866785,-0.858182,0.196922,-0.209174,-0.892119,-0.205637,-0.322458,0.567275,0.759375,0.482087,...,-0.303173,-0.478619,-0.429630,-0.486592,-0.449718,-0.887021,-0.010516,-0.051162,0.456813,-0.023092
75%,1.153688,1.165254,0.196922,0.396091,1.120927,-0.205637,-0.322458,0.567275,0.759375,0.482087,...,-0.303173,0.457294,-0.159827,-0.142474,-0.449718,1.127369,0.644317,0.963272,0.939638,0.496082
max,1.153688,1.165254,0.196922,10.534271,1.120927,4.862949,3.101183,0.567275,0.759375,0.482087,...,3.298445,2.329121,3.617407,2.954590,2.223615,1.127369,1.626566,0.963272,0.939638,11.303304


In [65]:
#defining the true labels

def df_to_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    df = {key: value.values[:, tf.newaxis] for key, value in df.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(y), labels))

    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))

    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)

    return ds


In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from ucimlrepo import fetch_ucirepo


inputs = {
    'HvyAlcoholConsump':
        tf.keras.layers.Input(
            shape=(1,),
            dtype=tf.float32,
            name='HvyAlcoholConsump'),
    'BMI':
        tf.keras.layers.Input(
            shape=(1,),
            dtype=tf.float32,
            name='BMI'),
    'SES_Variable':
        tf.keras.layers.Input(
            shape=(1,),
            dtype=tf.float32,
            name='SES_Variable'),
    'PhysHlth':
        tf.keras.layers.Input(
            shape=(1,),
            dtype=tf.float32,
            name='PhysHlth')
}

preprocessing_layers = tf.keras.layers.Concatenate()(
    [inputs.get('HvyAlcoholConsump'), inputs.get('BMI'), inputs.get('SES_Variable'), inputs.get('PhysHlth')])

hidden1 = tf.keras.layers.Dense(
    units=32, name='hidden1')(preprocessing_layers)

hidden2 = tf.keras.layers.Dense(
  units=8, name='hidden2')(hidden1)

dense_output = tf.keras.layers.Dense(
    units=1,
    name='dense_output')(hidden2)

# Define an output dictionary we'll send to the model constructor.
outputs = {
'dense_output': dense_output
}

dnn_model = tf.keras.Model(inputs=inputs, outputs=outputs)
dnn_model.compile(
    # ???
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    # Binary crossentropy
    loss=tf.keras.losses.BinaryCrossentropy(),
    # Precision, Recall, F1
    metrics=[tf.keras.metrics.F1Score()]
)

# Building the neural network and hidden layers, sigmoid activation fuction is used.
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model, using the adam optimizer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2) #80:20 ratio

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')




Epoch 1/10
5074/5074 [==============================] - 22s 4ms/step - loss: 0.3241 - accuracy: 0.8619 - val_loss: 0.3154 - val_accuracy: 0.8654
Epoch 2/10
5074/5074 [==============================] - 19s 4ms/step - loss: 0.3150 - accuracy: 0.8653 - val_loss: 0.3138 - val_accuracy: 0.8662
Epoch 3/10
5074/5074 [==============================] - 12s 2ms/step - loss: 0.3142 - accuracy: 0.8658 - val_loss: 0.3135 - val_accuracy: 0.8663
Epoch 4/10
5074/5074 [==============================] - 13s 2ms/step - loss: 0.3137 - accuracy: 0.8664 - val_loss: 0.3132 - val_accuracy: 0.8671
Epoch 5/10
5074/5074 [==============================] - 12s 2ms/step - loss: 0.3132 - accuracy: 0.8661 - val_loss: 0.3130 - val_accuracy: 0.8663
Epoch 6/10
5074/5074 [==============================] - 12s 2ms/step - loss: 0.3130 - accuracy: 0.8662 - val_loss: 0.3126 - val_accuracy: 0.8663
Epoch 7/10
5074/5074 [==============================] - 12s 2ms/step - loss: 0.3126 - accuracy: 0.8667 - val_loss: 0.3134 - val_ac

In [68]:
from sklearn.metrics import accuracy_score

y_pred_test = (model.predict(X_test) > 0.5).astype(int).flatten()

f1 = f1_score(y_test, y_pred_test, average='macro')
print(len(y_test), len(y_pred_test))
y_true = y['Diabetes_binary'].tolist()

accuracy = accuracy_score(y_test, y_pred_test)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

1586/1586 [==============================] - 4s 3ms/step
50736 50736
Accuracy: 0.8677
F1 Score: 0.5957


The training accuracy is also improving, reaching 86.73% after 10 epochs. The final validation accuracy is 86.6%. The test accuracy is around 86.7%. The consistency between training and validation accuracies suggests that the model is not overfitting the training data.The model achieves an accuracy of 86.77%. The F1 score is around 0.5957. The F1 score provides a balanced measure that considers both false positives and false negatives. F1's range is 0 to 1, the F1 that I got is .59 which is closer to 1 than 0 but overall average.